# Lecture 1 — OpenRouter API Testing

**Goal**: Explore different LLM models through OpenRouter API:

1. Check account credit balance
2. List available models with pricing
3. Compare outputs from Claude vs free/cheap models
4. Understand API request/response patterns
5. Observe differences in model capabilities

## Setup
This notebook requires:
- `OPENROUTER_API_KEY` (enter directly in Cell 1)



In [ ]:
# Add current directory to Python path for imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd()))


import json
from typing import Any, Dict, List
import pandas as pd

from openrouter_utils import (
    check_credits,
    print_remaining_credits,
    list_models,
    chat_completion,
    safe_chat,
    display_comparison
)

OPENROUTER_API_KEY = 'key'

# Models to test
MODELS = {
    "claude": "anthropic/claude-3.5-sonnet",
    "google-free": "google/gemma-3n-e2b-it:free",
    "qwen-free": "qwen/qwen3-4b:free",
    "mistral" : "mistralai/devstral-2512:free",
    "arcee": "arcee-ai/trinity-mini:free",
    "nvidia": "nvidia/nemotron-nano-9b-v2:free"
}

print("Imports loaded")
print(f"Testing {len(MODELS)} models:", list(MODELS.keys()))

if not OPENROUTER_API_KEY or OPENROUTER_API_KEY.strip() == "":
    raise RuntimeError(
        "⚠️  Please set OPENROUTER_API_KEY above before running this notebook.\n"
        "Get your key from: https://openrouter.ai/keys"
    )

print("✓ API key configured")

Imports loaded
Testing 6 models: ['claude', 'google-free', 'qwen-free', 'mistral', 'arcee', 'nvidia']
✓ API key configured


In [25]:
print_remaining_credits(OPENROUTER_API_KEY)

💳 API Key Credit Balance:
   Key limit:    $15.00
   Key usage:    $0.11
   Remaining:    $14.89


In [26]:
# Execute - function is imported from openrouter_utils.py
models_list = list_models(OPENROUTER_API_KEY, limit=100)

# Parse into DataFrame for easy viewing
models_df = pd.DataFrame([
    {
        "id": m.get("id", ""),
        "name": m.get("name", ""),
        "prompt_cost": m.get("pricing", {}).get("prompt", "N/A"),
        "completion_cost": m.get("pricing", {}).get("completion", "N/A"),
        "context_length": m.get("context_length", "N/A"),
    }
    for m in models_list
])

print(f"Found {len(models_df)} models")
print("\nOur test models:")
for key, model_id in MODELS.items():
    match = models_df[models_df["id"] == model_id]
    if not match.empty:
        row = match.iloc[0]
        print(f"  {key:10s} → ${row['prompt_cost']}/1M prompt tokens")
    else:
        print(f"  {key:10s} → {model_id} (not found in list)")

# Display sample of all models
models_df.head(20)

Found 100 models

Our test models:
  claude     → anthropic/claude-3.5-sonnet (not found in list)
  google-free → google/gemma-3n-e2b-it:free (not found in list)
  qwen-free  → qwen/qwen3-4b:free (not found in list)
  mistral    → $0/1M prompt tokens
  arcee      → $0/1M prompt tokens
  nvidia     → $0/1M prompt tokens


,id,name,prompt_cost,completion_cost,context_length
0,openai/gpt-5.2-codex,OpenAI: GPT-5.2-Codex,0.00000175,0.000014,400000
1,allenai/molmo-2-8b:free,AllenAI: Molmo2 8B (free),0,0,36864
2,allenai/olmo-3.1-32b-instruct,AllenAI: Olmo 3.1 32B Instruct,0.0000002,0.0000006,65536
3,bytedance-seed/seed-1.6-flash,ByteDance Seed: Seed 1.6 Flash,0.000000075,0.0000003,262144
4,bytedance-seed/seed-1.6,ByteDance Seed: Seed 1.6,0.00000025,0.000002,262144
5,minimax/minimax-m2.1,MiniMax: MiniMax M2.1,0.00000027,0.00000112,196608
6,z-ai/glm-4.7,Z.AI: GLM 4.7,0.0000004,0.0000015,202752
7,google/gemini-3-flash-preview,Google: Gemini 3 Flash Preview,0.0000005,0.000003,1048576
8,mistralai/mistral-small-creative,Mistral: Mistral Small Creative,0.0000001,0.0000003,32768
9,allenai/olmo-3.1-32b-think,AllenAI: Olmo 3.1 32B Think,0.00000015,0.0000005,65536


In [5]:
# Define test prompts that showcase model differences

PROMPTS = [
    {
        "name": "Factual Q&A",
        "messages": [
            {
                "role": "user",
                "content": "Explain what an API is in 2-3 sentences suitable for beginners."
            }
        ]
    },
    {
        "name": "Reasoning Task",
        "messages": [
            {
                "role": "user",
                "content": "If I have 3 apples and buy 2 more, then give away half, how many do I have left? Show your reasoning."
            }
        ]
    },
    {
        "name": "Code Explanation",
        "messages": [
            {
                "role": "user",
                "content": "Explain what this Python code does: `[x**2 for x in range(5)]`"
            }
        ]
    }
]



In [6]:
# Run all prompts through all models

results = []

for prompt_obj in PROMPTS:
    prompt_name = prompt_obj["name"]
    messages = prompt_obj["messages"]
    
    print(f"\n{'='*60}")
    print(f"Prompt: {prompt_name}")
    print(f"{'='*60}")
    
    for model_key, model_id in MODELS.items():
        print(f"\nTesting {model_key}...")
        
        result = chat_completion(
            OPENROUTER_API_KEY,
            model_id,
            messages,
            temperature=0.7,
            max_tokens=500
        )
        
        results.append({
            "prompt": prompt_name,
            "model_key": model_key,
            "model_id": model_id,
            **result
        })
        
        # Display output
        if result["error"]:
            print(f"  ❌ Error: {result['error']}")
        else:
            content = result["content"]
            preview = content
            print(f"  ✓ Response: {preview}")
            
            usage = result.get("usage", {})
            if usage:
                print(f"    Tokens: {usage.get('prompt_tokens', 0)} prompt + {usage.get('completion_tokens', 0)} completion")

print("\n✓ All tests complete")


Prompt: Factual Q&A

Testing claude...
  ✓ Response: An API (Application Programming Interface) is like a waiter at a restaurant - it takes requests and delivers responses between different software applications. It provides a set of rules and tools that allow different programs to communicate with each other, so for example, when you use a weather app, it's using an API to fetch weather data from a weather service's database.
    Tokens: 25 prompt + 75 completion

Testing google-free...
  ✓ Response: An API, or Application Programming Interface, is like a menu for software. It allows different software programs to talk to each other and share information without needing to know all the complicated details of how each program works.  Essentially, it's a set of rules and specifications that enables communication between applications.




    Tokens: 16 prompt + 62 completion

Testing qwen-free...
  ✓ Response: An API (Application Programming Interface) is a set of rules that allows dif

In [7]:
# Convert results to DataFrame for analysis

results_df = pd.DataFrame(results)

# Add computed fields
results_df["response_length"] = results_df["content"].str.len()
results_df["has_error"] = results_df["error"].notna()
results_df["total_tokens"] = results_df["usage"].apply(
    lambda u: u.get("total_tokens", 0) if isinstance(u, dict) else 0
)

print(f"Collected {len(results_df)} responses")
print(f"Errors: {results_df['has_error'].sum()}")

# Summary by model
summary = results_df.groupby("model_key").agg({
    "has_error": "sum",
    "response_length": "mean",
    "total_tokens": "sum"
}).round(1)

summary.columns = ["Errors", "Avg Response Length", "Total Tokens Used"]
print("\nSummary by Model:")
print(summary)

results_df.head()

Collected 18 responses
Errors: 0

Summary by Model:
             Errors  Avg Response Length  Total Tokens Used
model_key                                                  
arcee             0                350.7               1429
claude            0                351.3                441
google-free       0                633.3                558
mistral           0                643.7                665
nvidia            0                211.3               1598
qwen-free         0                256.0               3489


,prompt,model_key,model_id,model,content,parsed_content,usage,error,response_length,has_error,total_tokens
0,Factual Q&A,claude,anthropic/claude-3.5-sonnet,anthropic/claude-3.5-sonnet,An API (Application Programming Interface) is ...,None,"{'prompt_tokens': 25, 'completion_tokens': 75,...",None,377,False,100
1,Factual Q&A,google-free,google/gemma-3n-e2b-it:free,google/gemma-3n-e2b-it:free,"An API, or Application Programming Interface, ...",None,"{'prompt_tokens': 16, 'completion_tokens': 62,...",None,341,False,78
2,Factual Q&A,qwen-free,qwen/qwen3-4b:free,qwen/qwen3-4b:free,An API (Application Programming Interface) is ...,None,"{'prompt_tokens': 723, 'completion_tokens': 30...",None,401,False,1027
3,Factual Q&A,mistral,mistralai/devstral-2512:free,mistralai/devstral-2512:free,An **API (Application Programming Interface)**...,None,"{'prompt_tokens': 20, 'completion_tokens': 72,...",None,382,False,92
4,Factual Q&A,arcee,arcee-ai/trinity-mini:free,arcee-ai/trinity-mini:free,\n\nHere's a beginner-friendly explanation:\n\...,None,"{'prompt_tokens': 25, 'completion_tokens': 337...",None,618,False,362


In [8]:
# Function is now imported from openrouter_utils.py
# Display all comparisons
for prompt_obj in PROMPTS:
    display_comparison(results_df, prompt_obj["name"])


Prompt: Factual Q&A

[CLAUDE] (anthropic/claude-3.5-sonnet)
----------------------------------------------------------------------
An API (Application Programming Interface) is like a waiter at a restaurant - it takes requests and delivers responses between different software applications. It provides a set of rules and tools that allow different programs to communicate with each other, so for example, when you use a weather app, it's using an API to fetch weather data from a weather service's database.

Tokens: 100 total


[GOOGLE-FREE] (google/gemma-3n-e2b-it:free)
----------------------------------------------------------------------
An API, or Application Programming Interface, is like a menu for software. It allows different software programs to talk to each other and share information without needing to know all the complicated details of how each program works.  Essentially, it's a set of rules and specifications that enables communication between applications.





Tokens: 78 

In [9]:
# Structured Output Example: Extracting Laptop Specifications
# This demonstrates extracting structured data from unstructured text

laptop_description = """
Looking at this laptop listing, here's what I can tell you about it:

I've been looking at this Dell XPS 15 9520 and it's really impressive. It's their premium 15-inch model, 
and honestly, the display is gorgeous - it's a 15.6 inch OLED screen with 3840x2400 resolution, so it's 
super sharp. The panel type is OLED which gives amazing contrast and colors, way better than my old laptop.

For storage, the base model comes with a 512GB NVMe SSD, but the premium configuration I'm looking at 
has a 1TB NVMe SSD, which is pretty fast. The memory situation is interesting - the base model has 16GB 
of DDR5 RAM running at 4800MHz, while the premium version has 32GB. My current laptop only has 8GB, so 
even the base model would be twice the RAM I have now, which is pretty exciting. Both configurations can 
be upgraded to 64GB RAM if you need it later, which is nice for future-proofing.

Under the hood, it's powered by an Intel Core i7-12700H processor, which is a 12th gen chip with 14 cores. 
The graphics card is an NVIDIA GeForce RTX 3050 Ti with 4GB of dedicated VRAM. One really cool feature 
that caught my attention is that it's compatible with external GPU docks if you need more graphics power 
down the line - that's pretty forward-thinking.

Other features include: Wi-Fi 6E and Bluetooth 5.2 for connectivity, a 720p webcam (decent but not 
amazing), backlit keyboard which is always nice, and it weighs about 4.2 pounds - not the lightest but 
reasonable for a 15-inch. The battery is a 86Wh unit and they claim up to 10 hours of battery life, 
though I'd probably expect more like 7-8 in real use. It has 2 USB-C ports with Thunderbolt 4, 1 USB-A port, 
an HDMI 2.0 port, and a headphone jack. The operating system is Windows 11 Pro, and it's upgradeable to 
Windows 12 when that comes out, which is good to know.

The build quality is solid - it's made of aluminum and carbon fiber, feels really premium. The price was 
$1,899 last month when I first looked at it, but it's currently on sale for $1,599 which is a pretty good 
deal. Actually, I saw another listing that says it's 30% off the original $2,000 MSRP, which would make 
it $1,400, but that might be a different seller or refurbished model - I'm not entirely sure.

Overall, I think this is a solid choice, especially at the current sale price.
"""

# Define the extraction prompt with clear JSON schema
extraction_prompt = f"""Extract all laptop specifications from the following text and return them as structured JSON.

Text to analyze:
{laptop_description}

Extract the following information and return it as a JSON object with this exact structure:
{{
  "brand": "string (e.g., Dell, HP, Lenovo)",
  "model": "string (full model name/number)",
  "screen_size": number (in inches, e.g., 15.6),
  "screen_type": "string (e.g., OLED, IPS, LCD)",
  "resolution": "string (e.g., 3840x2400)",
  "storage": {{
    "capacity": number (in GB or TB, convert to GB),
    "type": "string (e.g., SSD, NVMe SSD, HDD)"
  }},
  "memory": {{
    "amount_gb": number (total RAM in GB),
    "type": "string (e.g., DDR4, DDR5)",
    "speed_mhz": number (if mentioned)
  }},
  "processor": {{
    "brand": "string (e.g., Intel, AMD)",
    "model": "string (full processor name)",
    "cores": number,
    "generation": "string or number (if mentioned)"
  }},
  "graphics": {{
    "type": "string (e.g., Integrated, Dedicated)",
    "brand": "string (e.g., NVIDIA, AMD, Intel)",
    "model": "string (e.g., RTX 3050 Ti)",
    "vram_gb": number (if dedicated GPU)
  }},
  "connectivity": {{
    "wifi": "string (e.g., Wi-Fi 6E, Wi-Fi 6)",
    "bluetooth": "string (e.g., Bluetooth 5.2)"
  }},
  "ports": {{
    "usb_c": number,
    "usb_a": number,
    "hdmi": number,
    "other": ["string"] (list other ports like headphone jack, etc.)
  }},
  "battery": {{
    "capacity_wh": number,
    "estimated_hours": number (if mentioned)
  }},
  "weight_lbs": number,
  "operating_system": "string",
  "webcam": "string (resolution if mentioned)",
  "keyboard": ["string"] (list features like backlit, etc.),
  "build_materials": ["string"] (list materials mentioned),
  "price": {{
    "regular": number (in USD, if mentioned),
    "sale": number (in USD, if on sale)
  }}
}}

Important:
- If a value is not mentioned in the text, use null for that field
- Convert all storage to GB (1TB = 1024GB)
- Extract exact numbers and strings as they appear
- Return ONLY valid JSON, no additional text or markdown formatting
"""

messages = [
    {"role": "user", "content": extraction_prompt}
]

output = {}
for model in MODELS:

    print(f"Running {model}")
    
    result = chat_completion(
        OPENROUTER_API_KEY,
        MODELS[model],
        messages,
        temperature=0.1,  # Very low temperature for consistent, accurate extraction
        max_tokens=800,   # Enough tokens for detailed JSON
        response_format={"type": "json_object"}  # Request JSON mode
    )

    output[model] = result

Running claude
Running google-free
Running qwen-free
Running mistral
Running arcee
Running nvidia


In [12]:
for model in output:
    if not output[model].get('parsed_content'):
        print(f"Model {model} has no structured output")
        continue
        
    #print(output[model]['parsed_content']['memory'])
    print(output[model]['parsed_content'])
    

{'brand': 'Dell', 'model': 'XPS 15 9520', 'screen_size': 15.6, 'screen_type': 'OLED', 'resolution': '3840x2400', 'storage': {'capacity': 1024, 'type': 'NVMe SSD'}, 'memory': {'amount_gb': 32, 'type': 'DDR5', 'speed_mhz': 4800}, 'processor': {'brand': 'Intel', 'model': 'Core i7-12700H', 'cores': 14, 'generation': 12}, 'graphics': {'type': 'Dedicated', 'brand': 'NVIDIA', 'model': 'GeForce RTX 3050 Ti', 'vram_gb': 4}, 'connectivity': {'wifi': 'Wi-Fi 6E', 'bluetooth': 'Bluetooth 5.2'}, 'ports': {'usb_c': 2, 'usb_a': 1, 'hdmi': 1, 'other': ['headphone jack']}, 'battery': {'capacity_wh': 86, 'estimated_hours': 10}, 'weight_lbs': 4.2, 'operating_system': 'Windows 11 Pro', 'webcam': '720p', 'keyboard': ['backlit'], 'build_materials': ['aluminum', 'carbon fiber'], 'price': {'regular': 2000, 'sale': 1599}}
Model google-free has no structured output
{'brand': 'Dell', 'model': 'XPS 15 9520', 'screen_size': 15.6, 'screen_type': 'OLED', 'resolution': '3840x2400', 'storage': {'capacity': 1024, 'type'

The information below contains a list of the highest level advertisting content categories used by advertisers:

Sports
Real Estate
Hobbies & Interests
Family and Relationships
Entertainment
Genres
Events
Personal Finance
Pets
Style & Fashion
Attractions
Law
Careers
Personal Celebrations & Life Events
Medical Health
Religion & Spirituality
Healthy Living
Food & Drink
Home & Garden
Communication
Pop Culture
Holidays
Technology & Computing
Automotive
Travel
Disasters
Fine Art
Sensitive Topics
Science
Books and Literature
Business and Finance
Politics
Shopping
War and Conflicts
Education
Video Gaming
Crime

for each of the websites below write code which takes the url in and tries to determine which category(s) it should be 

https://www.sfgate.com/hawaii/article/hawaii-eddie-aikau-surf-contest-21259632.php
https://www.bbc.com/news/articles/c041dp0z95yo
https://www.espn.com/nfl/playoff-bracket
https://www.motortrend.com/features/best-off-road-vehicles
https://www.allrecipes.com/gallery/best-canned-tuna-dinner-recipes/
https://www.goodreads.com/list/show/29490.Best_Romance_Books_of_2012
https://www.cancer.gov/about-cancer/treatment/side-effects/nutrition


In [20]:
prompt_2 = """
Classify each URL into the most relevant advertising category.

URLs to classify:
1. https://www.sfgate.com/hawaii/article/hawaii-eddie-aikau-surf-contest-21259632.php
2. https://www.bbc.com/news/articles/c041dp0z95yo
3. https://www.espn.com/nfl/playoff-bracket
4. https://www.motortrend.com/features/best-off-road-vehicles
5. https://www.allrecipes.com/gallery/best-canned-tuna-dinner-recipes/
6. https://www.goodreads.com/list/show/29490.Best_Romance_Books_of_2012
7. https://www.cancer.gov/about-cancer/treatment/side-effects/nutrition

Available categories:
Sports, Real Estate, Hobbies & Interests, Family and Relationships, Entertainment, Genres, Events, 
Personal Finance, Pets, Style & Fashion, Attractions, Law, Careers, Personal Celebrations & Life Events, 
Medical Health, Religion & Spirituality, Healthy Living, Food & Drink, Home & Garden, Communication, 
Pop Culture, Holidays, Technology & Computing, Automotive, Travel, Disasters, Fine Art, Sensitive Topics, 
Science, Books and Literature, Business and Finance, Politics, Shopping, War and Conflicts, Education, 
Video Gaming, Crime

Return a JSON object with this exact structure:
{
  "classifications": [
    {
      "url": "string (the URL)",
      "primary_category": "string (main category)",
      "secondary_categories": ["string"] (other relevant categories if any)
    }
  ]
}

Important:
- Each URL must have exactly one primary_category
- Only include secondary_categories if they are relevant
- Return ONLY valid JSON, no other text
"""

In [21]:

messages_2 = [
    {"role": "user", "content": prompt_2}
]

output_2 = {}
for model in MODELS:
    print(f"Running {model}")
    result = chat_completion(
        OPENROUTER_API_KEY,
        MODELS[model],
        messages_2,
        temperature=0.1,  # Very low temperature for consistent, accurate extraction
        max_tokens=800,   # Enough tokens for detailed JSON
        response_format={"type": "json_object"}  # Request JSON mode
    )
    output_2[model] = result



for model in output_2:
    if not output_2[model].get('parsed_content'):
        print(f"Model {model} has no structured output")
        continue
        
    print(f"\n{model}:")
    print(output_2[model]['parsed_content'])


Running claude
Running google-free
Running qwen-free
Running mistral
Running arcee
Running nvidia

claude:
{'classifications': [{'url': 'https://www.sfgate.com/hawaii/article/hawaii-eddie-aikau-surf-contest-21259632.php', 'primary_category': 'Sports', 'secondary_categories': ['Events']}, {'url': 'https://www.bbc.com/news/articles/c041dp0z95yo', 'primary_category': 'Sports', 'secondary_categories': []}, {'url': 'https://www.espn.com/nfl/playoff-bracket', 'primary_category': 'Sports', 'secondary_categories': ['Events']}, {'url': 'https://www.motortrend.com/features/best-off-road-vehicles', 'primary_category': 'Automotive', 'secondary_categories': []}, {'url': 'https://www.allrecipes.com/gallery/best-canned-tuna-dinner-recipes/', 'primary_category': 'Food & Drink', 'secondary_categories': []}, {'url': 'https://www.goodreads.com/list/show/29490.Best_Romance_Books_of_2012', 'primary_category': 'Books and Literature', 'secondary_categories': ['Genres']}, {'url': 'https://www.cancer.gov/about